In [1]:
import os
import pandas as pd
import re
from tqdm import tqdm
import pdb
import pickle

In [2]:
df1 = pd.read_excel('out_join.xlsx')
df1.head()

,Symbol,EndDate,Nshrttl,Nshrstt,Nshremp,Nshrsms,Stknme,Listexg,Conme,Nindcd,...,Explanation,IndustryCode,IndustryName,TotalAssets,TotalLiability,IntangibleAsset,NetProfit,OperatingEvenue,OperatingCost,OperationProfit
0,证券代码',统计截止日期',股本总数',其中：国有股',其中：内部职工股',其中：高管持股数',证券简称',上市交易所',公司中文全称',行业代码',...,说明',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,没有单位',没有单位',股',股',股',股',没有单位',没有单位',没有单位',没有单位',...,没有单位',没有单位',没有单位',元',元',元',元',元',元',元'
2,000001,2017-12-31,17170411366,0,0,23866,NaN,NaN,NaN,NaN,...,NaN,J66,货币金融服务,3248474000000,3026420000000,4701000000,23189000000,NaN,NaN,30223000000
3,000001,2016-12-31,17170411366,0,0,17587,NaN,NaN,NaN,NaN,...,NaN,J66,货币金融服务,2953434000000,2751263000000,4771000000,22599000000,NaN,NaN,29779000000
4,000001,2015-12-31,14308676139,0,0,14657,NaN,NaN,NaN,NaN,...,NaN,J66,货币金融服务,2507149000000,2345649000000,4961000000,21865000000,NaN,NaN,28895000000


In [6]:
df2 = pd.read_csv('catelog.csv',index_col=0)
print(df2.shape)
df2.head()

(11233, 4)


,code,id,title,url
0,703,000703_2017,恒逸石化：2017年年度报告,http://vip.stock.finance.sina.com.cn/corp/view...
1,703,000703_2016,恒逸石化：2016年年度报告,http://vip.stock.finance.sina.com.cn/corp/view...
2,703,000703_2015,恒逸石化：2015年年度报告（更新后）,http://vip.stock.finance.sina.com.cn/corp/view...
3,703,000703_2014,恒逸石化股份有限公司2014年年度报告,http://vip.stock.finance.sina.com.cn/corp/view...
4,703,000703_2013,恒逸石化股份有限公司2013年年度报告,http://vip.stock.finance.sina.com.cn/corp/view...


In [35]:
new_df = {'id':[],'size':[],'path':[]}
root = 'content'
for row in tqdm(df2.iterrows()):
    _id = row[1]['id']
    file = os.path.join(root,_id+'.txt')
    try:
        with open(file,'r',encoding='utf-8') as f:
            context = f.read()
    except:
        continue
    if len(context)<100:continue
    new_df['id']+=[_id]
    new_df['size'] += [len(context)]
    new_df['path'] += [file]

new_df = pd.DataFrame(new_df)
print(new_df.shape)
new_df.head()

11233it [00:33, 333.28it/s]


(10720, 3)


,id,path,size
0,000703_2017,content\000703_2017.txt,349485
1,000703_2014,content\000703_2014.txt,18954
2,000703_2013,content\000703_2013.txt,18912
3,000703_2012,content\000703_2012.txt,19137
4,000703_2011,content\000703_2011.txt,3726


In [36]:
new_df.to_csv('new_df.csv',index=False)

In [41]:
new_df = pd.read_csv('new_df.csv')

,id,path,size
0,000703_2017,content\000703_2017.txt,349485
1,000703_2014,content\000703_2014.txt,18954
2,000703_2013,content\000703_2013.txt,18912
3,000703_2012,content\000703_2012.txt,19137
4,000703_2011,content\000703_2011.txt,3726


In [42]:
new_df['code'] = new_df['id'].apply(lambda x: x[:6])
new_df['year'] = new_df['id'].apply(lambda x: x[-4:])
new_df.head()

,id,path,size,code,year
0,000703_2017,content\000703_2017.txt,349485,000703,2017
1,000703_2014,content\000703_2014.txt,18954,000703,2014
2,000703_2013,content\000703_2013.txt,18912,000703,2013
3,000703_2012,content\000703_2012.txt,19137,000703,2012
4,000703_2011,content\000703_2011.txt,3726,000703,2011


In [47]:
#filter year == 2018
new_df = new_df[new_df['year'] != '2018']
print(new_df.shape)
new_df.head()

(10719, 5)


,id,path,size,code,year
0,000703_2017,content\000703_2017.txt,349485,000703,2017
1,000703_2014,content\000703_2014.txt,18954,000703,2014
2,000703_2013,content\000703_2013.txt,18912,000703,2013
3,000703_2012,content\000703_2012.txt,19137,000703,2012
4,000703_2011,content\000703_2011.txt,3726,000703,2011


In [51]:
#get company which not has 2015 or 2016 or 2017
code_year = {}
for row in tqdm(new_df.iterrows()):
    _code = row[1]['code']
    _year = row[1]['year']
    if not _code in code_year: 
        code_year[_code] = [_year]
    else:
        code_year[_code] += [_year]
len(code_year)

10719it [00:00, 17853.33it/s]


1861

In [61]:
invalid_code = []
for k,v in code_year.items():
    if not '2015' in v or not '2016' in v or not '2017' in v: invalid_code += [k]
print(len(invalid_code))
print(invalid_code[:10])

815
['000703', '000791', '603196', '000013', '300063', '603660', '000405', '002898', '000699', '603937']


In [63]:
new_df[new_df['code']=='603660']

,id,path,size,code,year
64,603660_2017,content\603660_2017.txt,292409,603660,2017
65,603660_2016,content\603660_2016.txt,257691,603660,2016


In [70]:
new_df = new_df[~new_df['code'].isin(invalid_code)]

In [71]:
print(new_df.shape)
new_df.head(10)

(7475, 5)


,id,path,size,code,year
14,300193_2017,content\300193_2017.txt,290911,300193,2017
15,300193_2016,content\300193_2016.txt,283539,300193,2016
16,300193_2015,content\300193_2015.txt,19651,300193,2015
17,300193_2014,content\300193_2014.txt,18950,300193,2014
18,300193_2013,content\300193_2013.txt,18883,300193,2013
19,300193_2012,content\300193_2012.txt,18932,300193,2012
20,300193_2011,content\300193_2011.txt,3751,300193,2011
23,000701_2017,content\000701_2017.txt,491230,000701,2017
24,000701_2016,content\000701_2016.txt,487645,000701,2016
25,000701_2015,content\000701_2015.txt,19616,000701,2015


In [72]:
new_df.to_csv('new_df.csv',index=False)

In [75]:
new_df = new_df.rename(columns={'code':'Symbol'})
new_df.head()

,id,path,size,Symbol,year
14,300193_2017,content\300193_2017.txt,290911,300193,2017
15,300193_2016,content\300193_2016.txt,283539,300193,2016
16,300193_2015,content\300193_2015.txt,19651,300193,2015
17,300193_2014,content\300193_2014.txt,18950,300193,2014
18,300193_2013,content\300193_2013.txt,18883,300193,2013


In [86]:
freq_words = ['探索','搜寻','研究','设计','创新','变化','风险','实验','灵活','发现',
              '利用','提炼','精炼','选择','生产','效率','改进','选择','执行','实施']

print(len(freq_words))

20


In [98]:
temp = new_df[:2]

def fill_dict(file, freq_words_dict):
    with open(file, 'r', encoding='utf-8') as f:
        context = f.read()
    start = 0
    end = 2
    while(end < len(context)-1):
        word = context[start:end]
        if word in freq_words_dict: freq_words_dict[word]+=1
        start+=1
        end+=1
        
for row in temp.iterrows():
    freq_words_dict = {k:0 for k in freq_words}
    file = row[1]['path']
    print(file)
    fill_dict(file, freq_words_dict)
    print(freq_words_dict)
    


content\300193_2017.txt
{'探索': 0, '搜寻': 0, '研究': 21, '设计': 11, '创新': 18, '变化': 27, '风险': 113, '实验': 11, '灵活': 2, '发现': 25, '利用': 8, '提炼': 0, '精炼': 0, '选择': 7, '生产': 115, '效率': 7, '改进': 2, '执行': 27, '实施': 45}
content\300193_2016.txt
{'探索': 0, '搜寻': 0, '研究': 21, '设计': 8, '创新': 20, '变化': 18, '风险': 73, '实验': 6, '灵活': 2, '发现': 20, '利用': 7, '提炼': 0, '精炼': 0, '选择': 4, '生产': 108, '效率': 5, '改进': 3, '执行': 15, '实施': 45}


In [76]:
df1['year'] = df1['EndDate'].apply(lambda x:x[:4])
df1.head()

,Symbol,EndDate,Nshrttl,Nshrstt,Nshremp,Nshrsms,Stknme,Listexg,Conme,Nindcd,...,IndustryCode,IndustryName,TotalAssets,TotalLiability,IntangibleAsset,NetProfit,OperatingEvenue,OperatingCost,OperationProfit,year
0,证券代码',统计截止日期',股本总数',其中：国有股',其中：内部职工股',其中：高管持股数',证券简称',上市交易所',公司中文全称',行业代码',...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,统计截止
1,没有单位',没有单位',股',股',股',股',没有单位',没有单位',没有单位',没有单位',...,没有单位',没有单位',元',元',元',元',元',元',元',没有单位
2,000001,2017-12-31,17170411366,0,0,23866,NaN,NaN,NaN,NaN,...,J66,货币金融服务,3248474000000,3026420000000,4701000000,23189000000,NaN,NaN,30223000000,2017
3,000001,2016-12-31,17170411366,0,0,17587,NaN,NaN,NaN,NaN,...,J66,货币金融服务,2953434000000,2751263000000,4771000000,22599000000,NaN,NaN,29779000000,2016
4,000001,2015-12-31,14308676139,0,0,14657,NaN,NaN,NaN,NaN,...,J66,货币金融服务,2507149000000,2345649000000,4961000000,21865000000,NaN,NaN,28895000000,2015


In [81]:
df = df1.merge(new_df, how='inner')
print(df.shape)
df.head()

(7497, 60)


,Symbol,EndDate,Nshrttl,Nshrstt,Nshremp,Nshrsms,Stknme,Listexg,Conme,Nindcd,...,TotalLiability,IntangibleAsset,NetProfit,OperatingEvenue,OperatingCost,OperationProfit,year,id,path,size
0,000001,2017-12-31,17170411366,0,0,23866,NaN,NaN,NaN,NaN,...,3026420000000,4701000000,23189000000,NaN,NaN,30223000000,2017,000001_2017,content\000001_2017.txt,359787
1,000001,2016-12-31,17170411366,0,0,17587,NaN,NaN,NaN,NaN,...,2751263000000,4771000000,22599000000,NaN,NaN,29779000000,2016,000001_2016,content\000001_2016.txt,343535
2,000001,2015-12-31,14308676139,0,0,14657,NaN,NaN,NaN,NaN,...,2345649000000,4961000000,21865000000,NaN,NaN,28895000000,2015,000001_2015,content\000001_2015.txt,19655
3,000001,2014-12-31,11424894787,0,0,12213,NaN,NaN,NaN,NaN,...,2055510000000,5293000000,19802000000,NaN,NaN,26246000000,2014,000001_2014,content\000001_2014.txt,19192
4,000001,2013-12-31,8197360665,0,0,29782,NaN,NaN,NaN,NaN,...,1779660000000,5463000000,15231000000,NaN,NaN,19955000000,2013,000001_2013,content\000001_2013.txt,19439


In [82]:
df.to_csv('df.csv',index=False)